In [3]:
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, ClientsideFunction

import numpy as np
import pandas as pd
import pathlib
import plotly.express as px